In [24]:
"""
    @Author: Vipul Shete
    @Date: 2022-12-20
    @Last Modified by: Vipul Shete
    @Last Modified Date: 2022-12-20
    @Title : Cleaning of mobile and phone number in contact list (UC-1) .
"""

'\n    @Author: Vipul Shete\n    @Date: 2022-12-20\n    @Last Modified by: Vipul Shete\n    @Last Modified Date: 2022-12-20\n    @Title : Cleaning of mobile and phone number in contact list (UC-1) .\n'

In [56]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql.functions import concat, lit, col
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.types import *
import pyspark.sql.functions as f
from pyspark.sql.functions import col
import uuid

spark = SparkSession.builder.config("spark.jars", "/home/hdoop_new/CFP/AWS/CFP/aws/contact/MySqlJdbcDriver/mysql-connector-j-8.0.31.jar") \
    .master("local") \
    .appName("PySpark_MySQL_test3") \
    .getOrCreate()

sc=spark.sparkContext

In [108]:
data = spark.read.csv('contacts.csv', header=True)


In [109]:
data.show()

+----------+---------+------+-----------+----------+--------------------+----------+-------+
|first_name|last_name|  city|  mobile_no|  phone_no|               email|Is_email_s| Stream|
+----------+---------+------+-----------+----------+--------------------+----------+-------+
|     vipul|    shete|  beed| 8484891290| 702085447|vipulshete@gmail.com|       yes|   Mech|
|   vaibhav|    shete|mumbai|70208524447|9665281201|vaibhavshete@gmai...|        no|     IT|
|       ram|   shinde|  beed|     569856|      null| ramshinde@gmail.com|        no|  B.com|
|   shubham|    joshi|  pune| 1234567892|8484756892|   shubham@gmail.com|      null|    Law|
|   kailash|   jadhav|mumbai|       null|    598564|  pavanjad@gmail.com|       yes|     IT|
|     pavan|    rasal|  beed| 8484895260|4311222265|  pavanjad@gmail.com|      null|Teacher|
|      lisa|     dash|mumbai|    8569877|      null|  lisadash@gmail.com|       yes|     IT|
|   tukaram|   shinde|  pune|       null|8484891290|   tukaram@gmail.c

## encoding Is_email and branch

In [100]:
is_mail = spark.read.json("email_code.json")
mail = is_mail.collect()[0]

In [113]:
## replace value in IS_mail column

send_mail_send = data.withColumn("Is_mail_send", 
    f.when(data.Is_email_s=="yes", is_mail.collect()[0][1]) \
   .when(data.Is_email_s=="no", is_mail.collect()[0][0]) \
   .otherwise(data.Is_email_s)) 
send_mail_send.show()

+----------+---------+------+-----------+----------+--------------------+----------+-------+------------+
|first_name|last_name|  city|  mobile_no|  phone_no|               email|Is_email_s| Stream|Is_mail_send|
+----------+---------+------+-----------+----------+--------------------+----------+-------+------------+
|     vipul|    shete|  beed| 8484891290| 702085447|vipulshete@gmail.com|       yes|   Mech|           1|
|   vaibhav|    shete|mumbai|70208524447|9665281201|vaibhavshete@gmai...|        no|     IT|           0|
|       ram|   shinde|  beed|     569856|      null| ramshinde@gmail.com|        no|  B.com|           0|
|   shubham|    joshi|  pune| 1234567892|8484756892|   shubham@gmail.com|      null|    Law|        null|
|   kailash|   jadhav|mumbai|       null|    598564|  pavanjad@gmail.com|       yes|     IT|           1|
|     pavan|    rasal|  beed| 8484895260|4311222265|  pavanjad@gmail.com|      null|Teacher|        null|
|      lisa|     dash|mumbai|    8569877|     

In [103]:
stream = spark.read.json("stream_code.json")
stream_encoding = stream.collect()[0]
stream_encoding

Row(B.com='Non-Technical', CE='Technical', CS='Technical', EEE='Technical', IT='Technical', Law='Non-Technical', Mech='Technical', Teacher='Non-Technical')

In [114]:
## replace value in Branch column

stream_type = send_mail_send.withColumn("Type_of_strem", 
    f.when(data.Stream=="B.com", stream_encoding[0]) \
   .when(data.Stream=="CE", stream_encoding[1]) \
    .when(data.Stream=="CS", stream_encoding[2]) \
    .when(data.Stream=="EEE", stream_encoding[3]) \
    .when(data.Stream=="IT", stream_encoding[4]) \
    .when(data.Stream=="Law", stream_encoding[5]) \
    .when(data.Stream=="Mech", stream_encoding[6]) \
    .when(data.Stream=="Teacher", stream_encoding[7]) \
    .otherwise(data.Stream)) 
stream_type.show()

+----------+---------+------+-----------+----------+--------------------+----------+-------+------------+-------------+
|first_name|last_name|  city|  mobile_no|  phone_no|               email|Is_email_s| Stream|Is_mail_send|Type_of_strem|
+----------+---------+------+-----------+----------+--------------------+----------+-------+------------+-------------+
|     vipul|    shete|  beed| 8484891290| 702085447|vipulshete@gmail.com|       yes|   Mech|           1|    Technical|
|   vaibhav|    shete|mumbai|70208524447|9665281201|vaibhavshete@gmai...|        no|     IT|           0|    Technical|
|       ram|   shinde|  beed|     569856|      null| ramshinde@gmail.com|        no|  B.com|           0|Non-Technical|
|   shubham|    joshi|  pune| 1234567892|8484756892|   shubham@gmail.com|      null|    Law|        null|Non-Technical|
|   kailash|   jadhav|mumbai|       null|    598564|  pavanjad@gmail.com|       yes|     IT|           1|    Technical|
|     pavan|    rasal|  beed| 8484895260

In [116]:
# select columns

map_columns = stream_type.select("first_name", "last_name", "city", "mobile_no", "phone_no", "email", "Is_mail_send", "Stream", "Type_of_strem") 
map_columns.show()

+----------+---------+------+-----------+----------+--------------------+------------+-------+-------------+
|first_name|last_name|  city|  mobile_no|  phone_no|               email|Is_mail_send| Stream|Type_of_strem|
+----------+---------+------+-----------+----------+--------------------+------------+-------+-------------+
|     vipul|    shete|  beed| 8484891290| 702085447|vipulshete@gmail.com|           1|   Mech|    Technical|
|   vaibhav|    shete|mumbai|70208524447|9665281201|vaibhavshete@gmai...|           0|     IT|    Technical|
|       ram|   shinde|  beed|     569856|      null| ramshinde@gmail.com|           0|  B.com|Non-Technical|
|   shubham|    joshi|  pune| 1234567892|8484756892|   shubham@gmail.com|        null|    Law|Non-Technical|
|   kailash|   jadhav|mumbai|       null|    598564|  pavanjad@gmail.com|           1|     IT|    Technical|
|     pavan|    rasal|  beed| 8484895260|4311222265|  pavanjad@gmail.com|        null|Teacher|Non-Technical|
|      lisa|     da

In [117]:
mobile_no = map_columns.select("*", f.regexp_replace(f.col("mobile_no"), "[0/91]?[6789]{1}[0-9]{9}$", "True").alias("correct_mobile"))  
mobile_no.show()

+----------+---------+------+-----------+----------+--------------------+------------+-------+-------------+--------------+
|first_name|last_name|  city|  mobile_no|  phone_no|               email|Is_mail_send| Stream|Type_of_strem|correct_mobile|
+----------+---------+------+-----------+----------+--------------------+------------+-------+-------------+--------------+
|     vipul|    shete|  beed| 8484891290| 702085447|vipulshete@gmail.com|           1|   Mech|    Technical|          True|
|   vaibhav|    shete|mumbai|70208524447|9665281201|vaibhavshete@gmai...|           0|     IT|    Technical|   70208524447|
|       ram|   shinde|  beed|     569856|      null| ramshinde@gmail.com|           0|  B.com|Non-Technical|        569856|
|   shubham|    joshi|  pune| 1234567892|8484756892|   shubham@gmail.com|        null|    Law|Non-Technical|    1234567892|
|   kailash|   jadhav|mumbai|       null|    598564|  pavanjad@gmail.com|           1|     IT|    Technical|          null|
|     pa

In [118]:
mobile_no_1 = mobile_no.filter(mobile_no.correct_mobile=='True')
mobile_no_1.show()

+----------+---------+------+----------+----------+--------------------+------------+-------+-------------+--------------+
|first_name|last_name|  city| mobile_no|  phone_no|               email|Is_mail_send| Stream|Type_of_strem|correct_mobile|
+----------+---------+------+----------+----------+--------------------+------------+-------+-------------+--------------+
|     vipul|    shete|  beed|8484891290| 702085447|vipulshete@gmail.com|           1|   Mech|    Technical|          True|
|     pavan|    rasal|  beed|8484895260|4311222265|  pavanjad@gmail.com|        null|Teacher|Non-Technical|          True|
|    smarth|    shete|mumbai|8756987845|8484892448|    smarth@gmail.com|           1|    EEE|    Technical|          True|
|      renu|    gonde|  beed|8484891290|      null| renugonde@gmail.com|           0|     IT|    Technical|          True|
+----------+---------+------+----------+----------+--------------------+------------+-------+-------------+--------------+



In [119]:
validate_mobile = mobile_no_1.select('first_name', 'last_name', 'city', 'mobile_no', 'email', "Is_mail_send", "Stream", "Type_of_strem")
validate_mobile.show()

+----------+---------+------+----------+--------------------+------------+-------+-------------+
|first_name|last_name|  city| mobile_no|               email|Is_mail_send| Stream|Type_of_strem|
+----------+---------+------+----------+--------------------+------------+-------+-------------+
|     vipul|    shete|  beed|8484891290|vipulshete@gmail.com|           1|   Mech|    Technical|
|     pavan|    rasal|  beed|8484895260|  pavanjad@gmail.com|        null|Teacher|Non-Technical|
|    smarth|    shete|mumbai|8756987845|    smarth@gmail.com|           1|    EEE|    Technical|
|      renu|    gonde|  beed|8484891290| renugonde@gmail.com|           0|     IT|    Technical|
+----------+---------+------+----------+--------------------+------------+-------+-------------+



In [121]:
phone_no = map_columns.select('first_name', 'last_name', 'city', 'phone_no', 'email', "Is_mail_send", "Stream", "Type_of_strem")
phone_no.show()

+----------+---------+------+----------+--------------------+------------+-------+-------------+
|first_name|last_name|  city|  phone_no|               email|Is_mail_send| Stream|Type_of_strem|
+----------+---------+------+----------+--------------------+------------+-------+-------------+
|     vipul|    shete|  beed| 702085447|vipulshete@gmail.com|           1|   Mech|    Technical|
|   vaibhav|    shete|mumbai|9665281201|vaibhavshete@gmai...|           0|     IT|    Technical|
|       ram|   shinde|  beed|      null| ramshinde@gmail.com|           0|  B.com|Non-Technical|
|   shubham|    joshi|  pune|8484756892|   shubham@gmail.com|        null|    Law|Non-Technical|
|   kailash|   jadhav|mumbai|    598564|  pavanjad@gmail.com|           1|     IT|    Technical|
|     pavan|    rasal|  beed|4311222265|  pavanjad@gmail.com|        null|Teacher|Non-Technical|
|      lisa|     dash|mumbai|      null|  lisadash@gmail.com|           1|     IT|    Technical|
|   tukaram|   shinde|  pune|8

In [122]:
phone_no_1=phone_no.withColumnRenamed("phone_no","mobile_no")
phone_no_1.show()

+----------+---------+------+----------+--------------------+------------+-------+-------------+
|first_name|last_name|  city| mobile_no|               email|Is_mail_send| Stream|Type_of_strem|
+----------+---------+------+----------+--------------------+------------+-------+-------------+
|     vipul|    shete|  beed| 702085447|vipulshete@gmail.com|           1|   Mech|    Technical|
|   vaibhav|    shete|mumbai|9665281201|vaibhavshete@gmai...|           0|     IT|    Technical|
|       ram|   shinde|  beed|      null| ramshinde@gmail.com|           0|  B.com|Non-Technical|
|   shubham|    joshi|  pune|8484756892|   shubham@gmail.com|        null|    Law|Non-Technical|
|   kailash|   jadhav|mumbai|    598564|  pavanjad@gmail.com|           1|     IT|    Technical|
|     pavan|    rasal|  beed|4311222265|  pavanjad@gmail.com|        null|Teacher|Non-Technical|
|      lisa|     dash|mumbai|      null|  lisadash@gmail.com|           1|     IT|    Technical|
|   tukaram|   shinde|  pune|8

In [123]:
phone_no_2 = phone_no_1.select("*", f.regexp_replace(f.col("mobile_no"), "[0/91]?[6789]{1}[0-9]{9}$", "True").alias("correct_mobile"))  
phone_no_2.show()

+----------+---------+------+----------+--------------------+------------+-------+-------------+--------------+
|first_name|last_name|  city| mobile_no|               email|Is_mail_send| Stream|Type_of_strem|correct_mobile|
+----------+---------+------+----------+--------------------+------------+-------+-------------+--------------+
|     vipul|    shete|  beed| 702085447|vipulshete@gmail.com|           1|   Mech|    Technical|     702085447|
|   vaibhav|    shete|mumbai|9665281201|vaibhavshete@gmai...|           0|     IT|    Technical|          True|
|       ram|   shinde|  beed|      null| ramshinde@gmail.com|           0|  B.com|Non-Technical|          null|
|   shubham|    joshi|  pune|8484756892|   shubham@gmail.com|        null|    Law|Non-Technical|          True|
|   kailash|   jadhav|mumbai|    598564|  pavanjad@gmail.com|           1|     IT|    Technical|        598564|
|     pavan|    rasal|  beed|4311222265|  pavanjad@gmail.com|        null|Teacher|Non-Technical|    4311

In [124]:
phone_no_3 = phone_no_2.filter(phone_no_2.correct_mobile=='True')
phone_no_3.show()

+----------+---------+------+----------+--------------------+------------+------+-------------+--------------+
|first_name|last_name|  city| mobile_no|               email|Is_mail_send|Stream|Type_of_strem|correct_mobile|
+----------+---------+------+----------+--------------------+------------+------+-------------+--------------+
|   vaibhav|    shete|mumbai|9665281201|vaibhavshete@gmai...|           0|    IT|    Technical|          True|
|   shubham|    joshi|  pune|8484756892|   shubham@gmail.com|        null|   Law|Non-Technical|          True|
|   tukaram|   shinde|  pune|8484891290|   tukaram@gmail.com|           0|    CS|    Technical|          True|
|    smarth|    shete|mumbai|8484892448|    smarth@gmail.com|           1|   EEE|    Technical|          True|
+----------+---------+------+----------+--------------------+------------+------+-------------+--------------+



In [125]:
validate_phone = phone_no_3.select('first_name', 'last_name', 'city', 'mobile_no', 'email', "Is_mail_send", "Stream", "Type_of_strem")
validate_phone.show()

+----------+---------+------+----------+--------------------+------------+------+-------------+
|first_name|last_name|  city| mobile_no|               email|Is_mail_send|Stream|Type_of_strem|
+----------+---------+------+----------+--------------------+------------+------+-------------+
|   vaibhav|    shete|mumbai|9665281201|vaibhavshete@gmai...|           0|    IT|    Technical|
|   shubham|    joshi|  pune|8484756892|   shubham@gmail.com|        null|   Law|Non-Technical|
|   tukaram|   shinde|  pune|8484891290|   tukaram@gmail.com|           0|    CS|    Technical|
|    smarth|    shete|mumbai|8484892448|    smarth@gmail.com|           1|   EEE|    Technical|
+----------+---------+------+----------+--------------------+------------+------+-------------+



In [126]:
validate_number = validate_mobile.union(validate_phone)
validate_number.show()

+----------+---------+------+----------+--------------------+------------+-------+-------------+
|first_name|last_name|  city| mobile_no|               email|Is_mail_send| Stream|Type_of_strem|
+----------+---------+------+----------+--------------------+------------+-------+-------------+
|     vipul|    shete|  beed|8484891290|vipulshete@gmail.com|           1|   Mech|    Technical|
|     pavan|    rasal|  beed|8484895260|  pavanjad@gmail.com|        null|Teacher|Non-Technical|
|    smarth|    shete|mumbai|8756987845|    smarth@gmail.com|           1|    EEE|    Technical|
|      renu|    gonde|  beed|8484891290| renugonde@gmail.com|           0|     IT|    Technical|
|   vaibhav|    shete|mumbai|9665281201|vaibhavshete@gmai...|           0|     IT|    Technical|
|   shubham|    joshi|  pune|8484756892|   shubham@gmail.com|        null|    Law|Non-Technical|
|   tukaram|   shinde|  pune|8484891290|   tukaram@gmail.com|           0|     CS|    Technical|
|    smarth|    shete|mumbai|8

In [127]:
## unique mobile number

drop_duplicate_number = validate_number.drop_duplicates(["mobile_no"])
drop_duplicate_number.show()


+----------+---------+------+----------+--------------------+------------+-------+-------------+
|first_name|last_name|  city| mobile_no|               email|Is_mail_send| Stream|Type_of_strem|
+----------+---------+------+----------+--------------------+------------+-------+-------------+
|   shubham|    joshi|  pune|8484756892|   shubham@gmail.com|        null|    Law|Non-Technical|
|     vipul|    shete|  beed|8484891290|vipulshete@gmail.com|           1|   Mech|    Technical|
|    smarth|    shete|mumbai|8484892448|    smarth@gmail.com|           1|    EEE|    Technical|
|     pavan|    rasal|  beed|8484895260|  pavanjad@gmail.com|        null|Teacher|Non-Technical|
|    smarth|    shete|mumbai|8756987845|    smarth@gmail.com|           1|    EEE|    Technical|
|   vaibhav|    shete|mumbai|9665281201|vaibhavshete@gmai...|           0|     IT|    Technical|
+----------+---------+------+----------+--------------------+------------+-------+-------------+



In [128]:
country_code=drop_duplicate_number.withColumn("country_code", f.lit('+91'))
country_code.show()


+----------+---------+------+----------+--------------------+------------+-------+-------------+------------+
|first_name|last_name|  city| mobile_no|               email|Is_mail_send| Stream|Type_of_strem|country_code|
+----------+---------+------+----------+--------------------+------------+-------+-------------+------------+
|   shubham|    joshi|  pune|8484756892|   shubham@gmail.com|        null|    Law|Non-Technical|         +91|
|     vipul|    shete|  beed|8484891290|vipulshete@gmail.com|           1|   Mech|    Technical|         +91|
|    smarth|    shete|mumbai|8484892448|    smarth@gmail.com|           1|    EEE|    Technical|         +91|
|     pavan|    rasal|  beed|8484895260|  pavanjad@gmail.com|        null|Teacher|Non-Technical|         +91|
|    smarth|    shete|mumbai|8756987845|    smarth@gmail.com|           1|    EEE|    Technical|         +91|
|   vaibhav|    shete|mumbai|9665281201|vaibhavshete@gmai...|           0|     IT|    Technical|         +91|
+---------

In [129]:
final_contacts=country_code.select("*", concat(col("country_code"),lit(" "),col("mobile_no")).alias("countery_code_with_mobile_no"))
final_contacts.show()

+----------+---------+------+----------+--------------------+------------+-------+-------------+------------+----------------------------+
|first_name|last_name|  city| mobile_no|               email|Is_mail_send| Stream|Type_of_strem|country_code|countery_code_with_mobile_no|
+----------+---------+------+----------+--------------------+------------+-------+-------------+------------+----------------------------+
|   shubham|    joshi|  pune|8484756892|   shubham@gmail.com|        null|    Law|Non-Technical|         +91|              +91 8484756892|
|     vipul|    shete|  beed|8484891290|vipulshete@gmail.com|           1|   Mech|    Technical|         +91|              +91 8484891290|
|    smarth|    shete|mumbai|8484892448|    smarth@gmail.com|           1|    EEE|    Technical|         +91|              +91 8484892448|
|     pavan|    rasal|  beed|8484895260|  pavanjad@gmail.com|        null|Teacher|Non-Technical|         +91|              +91 8484895260|
|    smarth|    shete|mumba

In [130]:
contacts = final_contacts.select('first_name', 'last_name', 'city', 'countery_code_with_mobile_no', 'email', "Is_mail_send", "Stream", "Type_of_strem")
contacts.show(truncate=False)

+----------+---------+------+----------------------------+----------------------+------------+-------+-------------+
|first_name|last_name|city  |countery_code_with_mobile_no|email                 |Is_mail_send|Stream |Type_of_strem|
+----------+---------+------+----------------------------+----------------------+------------+-------+-------------+
|shubham   |joshi    |pune  |+91 8484756892              |shubham@gmail.com     |null        |Law    |Non-Technical|
|vipul     |shete    |beed  |+91 8484891290              |vipulshete@gmail.com  |1           |Mech   |Technical    |
|smarth    |shete    |mumbai|+91 8484892448              |smarth@gmail.com      |1           |EEE    |Technical    |
|pavan     |rasal    |beed  |+91 8484895260              |pavanjad@gmail.com    |null        |Teacher|Non-Technical|
|smarth    |shete    |mumbai|+91 8756987845              |smarth@gmail.com      |1           |EEE    |Technical    |
|vaibhav   |shete    |mumbai|+91 9665281201              |vaibha

In [131]:
try:
    id = []
    for i in range(contacts.count()):
        id.append(str(uuid.uuid4()))
    contact_df = contacts.repartition(1).withColumn('id',f.udf(lambda x: id[x])(f.monotonically_increasing_id()))

    contact_df = contact_df.select('id','first_name','last_name','city','countery_code_with_mobile_no','email')
    contact_df.show(truncate=False)
except Exception as e:
    print(e)

+------------------------------------+----------+---------+------+----------------------------+----------------------+
|id                                  |first_name|last_name|city  |countery_code_with_mobile_no|email                 |
+------------------------------------+----------+---------+------+----------------------------+----------------------+
|c5cbaa6d-7dd8-4c76-9713-264623264158|shubham   |joshi    |pune  |+91 8484756892              |shubham@gmail.com     |
|049068a4-cec5-4a87-8104-25e07cf2416e|vipul     |shete    |beed  |+91 8484891290              |vipulshete@gmail.com  |
|c60ccb4a-3fbe-475f-a40d-f17959c38766|smarth    |shete    |mumbai|+91 8484892448              |smarth@gmail.com      |
|3c663512-8b1a-454e-8a04-662dc7ca09a8|pavan     |rasal    |beed  |+91 8484895260              |pavanjad@gmail.com    |
|e399da24-1e97-48a3-97c7-78cfda67cf1a|smarth    |shete    |mumbai|+91 8756987845              |smarth@gmail.com      |
|254d5c7d-0cd2-4a10-a6b4-861e8862bfa2|vaibhav   

## load table to mysql database

In [42]:
contact_df.write.format('jdbc').options(
      url='jdbc:mysql://localhost/my_first_db',
      driver='com.mysql.jdbc.Driver',
      dbtable='contacts',
      user='root',
      password='Vaibhav@123').mode('append').save()